# Introduction to Bedrock - Fine-Tuning

> *If you see errors, you may need to be allow-listed for the Bedrock models used by this notebook*

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*


In this demo notebook, we demonstrate how to use the Bedrock Python SDK for fine-tuning Bedrock models with your own data. If you have text samples to train and want to adapt the Bedrock models to your domain, you can further fine-tune the Bedrock foundation models by providing your own training datasets. You can upload your datasets to Amazon S3, and provide the S3 bucket path while configuring a Bedrock fine-tuning job. You can also adjust hyper parameters (learning rate, epoch, and batch size) for fine-tuning. After the fine-tuning job of the model with your dataset has completed, you can start using the model for inference in the Bedrock playground application. You can select the fine-tuned model and submit a prompt to the fine-tuned model along with a set of model parameters. The fine-tuned model should generate texts to be more alike your text samples. 

-----------

1. Setup
2. Fine-tuning
3. Testing the fine-tuned model

 Note: This notebook was tested in Amazon SageMaker Studio with Python 3 (Data Science 2.0) kernel.

---

## 1. Setup

In [ ]:
%pip install -U boto3 botocore --force-reinstall --quiet

In [ ]:
%pip list | grep boto3

#### Now let's set up our connection to the Amazon Bedrock SDK using Boto3

In [ ]:
#### Un comment the following lines to run from your local environment outside of the AWS account with Bedrock access

#import os
#os.environ['BEDROCK_ASSUME_ROLE'] = '<YOUR_VALUES>'
#os.environ['AWS_PROFILE'] = '<YOUR_VALUES>'

In [ ]:
import boto3
import json
import os
import sys

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww

bedrock_client = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
    runtime=False  # Needed for control plane
)

In [ ]:
#bedrock_client.list_foundation_models()

In [ ]:
import sagemaker

sess = sagemaker.Session()
sagemaker_session_bucket = sess.default_bucket()
role = sagemaker.get_execution_role()

### Preview custom data

Our custom data in JSON line file format.

In [ ]:
data = "data/train.jsonl"

Read the JSON line file into an object like any normal file

In [ ]:
with open(data) as f:
    lines = f.read().splitlines()

#### Load the ‘lines’ object into a pandas Data Frame.

In [ ]:
import pandas as pd
df_inter = pd.DataFrame(lines)
df_inter.columns = ['json_element']

This intermediate data frame will have only one column with each json object in a row. A sample output is given below.

In [ ]:
df_inter['json_element'].apply(json.loads)

Now we will apply json loads function on each row of the ‘json_element’ column. ‘json.loads’ is a decoder function in python which is used to decode a json object into a dictionary. ‘apply’ is a popular function in pandas that takes any function and applies to each row of the pandas dataframe or series.

In [ ]:
df_final = pd.json_normalize(df_inter['json_element'].apply(json.loads))

Once decoding is done we will apply the json normalize function to the above result. json normalize will convert any semi-structured json data into a flat table. Here it converts the JSON ‘keys’ to columns and its corresponding values to row elements.

In [ ]:
df_final

### Uploading data to S3

Next, we need to upload our training dataset to S3:

In [ ]:
s3_location = f"s3://{sagemaker_session_bucket}/bedrock/finetuning/train.jsonl"
s3_output = f"s3://{sagemaker_session_bucket}/bedrock/finetuning/output"

In [ ]:
!aws s3 cp data/train.jsonl $s3_location

Now we can create the fine-tuning job. 

### ^^ **Note:** Make sure the IAM role you're using has these [IAM policies](https://docs.aws.amazon.com/bedrock/latest/userguide/model-customization-iam-role.html) attached that allow Amazon Bedrock access to the specified S3 buckets ^^

## 2. Fine-tuning

In [ ]:
import time
timestamp = int(time.time())

In [ ]:
base_model_id = "amazon.titan-text-express-v1"

In [ ]:
job_name = "titan-{}".format(timestamp)
job_name

In [ ]:
custom_model_name = "custom-{}".format(job_name)
custom_model_name

In [ ]:
bedrock_client.create_model_customization_job(
    jobName=job_name,
    customModelName=custom_model_name,
    roleArn=role,
    baseModelIdentifier=base_model_id,
    hyperParameters = {
        "epochCount": "1",
        "batchSize": "1",
        "learningRate": "0.005",
        "learningRateWarmupSteps": "0"
    },
    trainingDataConfig={"s3Uri": s3_location},
    outputDataConfig={"s3Uri": s3_output},
)

Let's periodically check in on the progress. The training job should take between 5 and 10 minutes.

In [ ]:
status = bedrock_client.get_model_customization_job(jobIdentifier=job_name)["status"]
status

## The next cell might run for ~40min

In [ ]:
import time

status = bedrock_client.get_model_customization_job(jobIdentifier=job_name)["status"]

while status == "InProgress":
    print(status)
    time.sleep(30)
    status = bedrock_client.get_model_customization_job(jobIdentifier=job_name)["status"]
    
print(status)

In [ ]:
completed_job = bedrock_client.get_model_customization_job(jobIdentifier=job_name)
completed_job

## 3. Testing

Now we can test the fine-tuned model

In [ ]:
bedrock_client.list_custom_models()

In [ ]:
for job in bedrock_client.list_model_customization_jobs()["modelCustomizationJobSummaries"]:
    print("-----\n" + "jobArn: " + job["jobArn"] + "\njobName: " + job["jobName"] + "\nstatus: " + job["status"] + "\ncustomModelName: " + job["customModelName"])

## GetCustomModel

In [ ]:
bedrock_client.get_custom_model(modelIdentifier=custom_model_name)

In [ ]:
custom_model_arn = bedrock_client.get_custom_model(modelIdentifier=custom_model_name)['modelArn']
custom_model_arn

In [ ]:
base_model_arn = bedrock_client.get_custom_model(modelIdentifier=custom_model_name)['baseModelArn']
base_model_arn

## **Note:** To invoke custom models, you need to first create a provisioned throughput resource and make requests using that resource.

In [ ]:
provisioned_model_name = "pt-{}".format(custom_model_name)
provisioned_model_name

## !! **Note:** SDK currently only supports 1 month and 6 months commitment terms. Go to Bedrock console to manually purchase no commitment term option for testing !!

In [ ]:
# bedrock_client.create_provisioned_model_throughput(
#     modelUnits = 1,
#     commitmentDuration = "OneMonth", ## Note: SDK is currently missing No Commitment option
#     provisionedModelName = provisioned_model_name,
#     modelId = base_model_arn
# ) 

## ListProvisionedModelThroughputs

In [ ]:
bedrock_client.list_provisioned_model_throughputs()["provisionedModelSummaries"]

## GetProvisionedModelThroughput

In [ ]:
provisioned_model_name = "pt-custom-titan-1697927162"

In [ ]:
provisioned_model_arn = bedrock_client.get_provisioned_model_throughput(
     provisionedModelId=provisioned_model_name)["provisionedModelArn"]
provisioned_model_arn

In [ ]:
deployment_status = bedrock_client.get_provisioned_model_throughput(
    provisionedModelId=provisioned_model_name)["status"]
deployment_status

## The next cell might run for ~10min

In [ ]:
import time

deployment_status = bedrock_client.get_provisioned_model_throughput(
    provisionedModelId=provisioned_model_name)["status"]

while deployment_status == "Creating":
    
    print(deployment_status)
    time.sleep(30)
    deployment_status = bedrock_client.get_provisioned_model_throughput(
        provisionedModelId=provisioned_model_name)["status"]  
    
print(deployment_status)

### Invoke Model

In [ ]:
import boto3
import json
import os
import sys

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww

bedrock_client = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
    runtime=True  # Default. Needed for invoke_model() on the data plane
)

In [ ]:
response = bedrock_client.invoke_model(
    # modelId needs to be Provisioned Throughput Model ARN
    modelId=provisioned_model_arn,
    body="{\"inputText\":\"I really don't like this!\",\"textGenerationConfig\":{\"maxTokenCount\":50,\"stopSequences\":[],\"temperature\":1,\"topP\":0.9}}"
)

print(response)

### Parse Model Response Body

In [ ]:
response_body = response["body"].read().decode('utf8')
response_body

### Show OutputText

In [ ]:
json.loads(response_body)["results"][0]["outputText"]

## Delete Provisioned Throughput

When you're done testing, you can delete Provisioned Throughput to stop charges

In [ ]:
# bedrock_client = bedrock.get_bedrock_client(
#     assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
#     region=os.environ.get("AWS_DEFAULT_REGION", None),
#     runtime=False  # Default. Needed for invoke_model() on the data plane
# )

In [ ]:
# bedrock_client.delete_provisioned_model_throughput(
#     provisionedModelId = provisioned_model_name
# )